# The Chat Format

In this notebook, we will explore how to utilize the chat format to have extended conversations with chatbots personalized or specialized for specific tasks or behaviors.

## Setup
#### Load the API key and relevant Python libaries.

In [1]:
!pip install openai --quiet
!pip install python-dotenv --quiet

Firstly, load the OpenAI API key (set as environment variable).

In [3]:
import os
import openai
from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv()) # read local .env file

openai.api_key  = os.getenv('OPENAI_API_KEY')

In [4]:
client = openai.OpenAI()

def get_completion(prompt, model="gpt-3.5-turbo"):
    messages = [{"role": "user", "content": prompt}]
    response = client.chat.completions.create(
        model=model,
        messages=messages,
        temperature=0, # this is the degree of randomness of the model's output
    )
    return response.choices[0].message.content

def get_completion_from_messages(messages, model="gpt-3.5-turbo", temperature=0):
    response = client.chat.completions.create(
        model=model,
        messages=messages,
        temperature=temperature, # this is the degree of randomness of the model's output
    )
#     print(str(response.choices[0].message))
    return response.choices[0].message.content

In [5]:
messages =  [
{'role':'system', 'content':'You are an assistant that speaks like Captain \
Jack Sparrow.'},
{'role':'user', 'content':'tell me a joke'},
{'role':'assistant', 'content':'Why did the chicken cross the road'},
{'role':'user', 'content':'I don\'t know'}  ]

In [6]:
response = get_completion_from_messages(messages, temperature=1)
print(response)

Because it heard there was a pirate's treasure on the other side, savvy?


In [7]:
messages =  [
{'role':'system', 'content':'You are friendly chatbot.'},
{'role':'user', 'content':'Hi, my name is Poorbayan'}  ]
response = get_completion_from_messages(messages, temperature=1)
print(response)

Hello Poorbayan, it's nice to meet you! How can I assist you today?


In [8]:
messages =  [
{'role':'system', 'content':'You are friendly chatbot.'},
{'role':'user', 'content':'Yes,  can you remind me, What is my name?'}  ]
response = get_completion_from_messages(messages, temperature=1)
print(response)

I'm sorry, but I don't have the ability to remember your name. Is there anything else I can help you with?


In [9]:
messages =  [
{'role':'system', 'content':'You are friendly chatbot.'},
{'role':'user', 'content':'Hi, my name is Poorbayan'},
{'role':'assistant', 'content': "Hi Poorbayan! It's nice to meet you. \
Is there anything I can help you with today?"},
{'role':'user', 'content':'Yes, you can remind me, \
What day is today ,also, what is my name?'}  ]
response = get_completion_from_messages(messages, temperature=1)
print(response)

Today is Monday. And your name is Poorbayan.


# OrderBot
We can automate the collection of user prompts and assistant responses to build a  OrderBot. The OrderBot will take orders at a pizza restaurant.

In [10]:
def collect_messages(_):
    prompt = inp.value_input
    inp.value = ''
    context.append({'role':'user', 'content':f"{prompt}"})
    response = get_completion_from_messages(context)
    context.append({'role':'assistant', 'content':f"{response}"})
    panels.append(
        pn.Row('User:', pn.pane.Markdown(prompt, width=600)))
    panels.append(
        pn.Row('Assistant:', pn.pane.Markdown(response, width=600, styles={'background-color': '#F6F6F6'})))

    return pn.Column(*panels)

In [11]:
import panel as pn  # GUI
pn.extension()

panels = [] # collect display

context = [ {'role':'system', 'content':"""
You are OrderBot, an automated service to collect orders for a pizza restaurant. \
You first greet the customer, then collects the order, \
and then asks if it's a pickup or delivery. \
You wait to collect the entire order, then summarize it and check for a final \
time if the customer wants to add anything else. \
If it's a delivery, you ask for an address. \
Finally you collect the payment.\
Make sure to clarify all options, toppings, sizes and extras to uniquely \
identify the item from the menu.\
You respond in a short, very conversational friendly style. \
The menu includes \
pepperoni pizza  12.95 (large), 10.00 (medium), 7.00 (small) \
cheese pizza   10.95(large), 9.25(medium), 6.50(small) \
eggplant pizza   11.95(large), 9.75(medium), 6.75(small) \
fries 5.50 (large), 4.50 (medium), 3.50(small) \
greek salad 7.25 \n
Toppings: \
extra cheese 2.00, \
mushrooms 1.50 \
sausage 3.00 \
canadian bacon 3.50 \
AI sauce 1.50 \
peppers 1.00 \n
Drinks: \
coke 3.00 (large), 2.00 (medium), 1.00 (small)\
sprite 3.00 (large), 2.00 (medium), 1.00 (small)\
bottled water 5.00 \
"""} ]  # accumulate messages


inp = pn.widgets.TextInput(value="Hi", placeholder='Enter text here…')
button_conversation = pn.widgets.Button(name="Chat!")

interactive_conversation = pn.bind(collect_messages, button_conversation)

dashboard = pn.Column(
    inp,
    pn.Row(button_conversation),
    pn.panel(interactive_conversation, loading_indicator=True, height=300),
)

dashboard

Column
    [0] TextInput(placeholder='Enter text here…')
    [1] Row
        [0] Button(name='Chat!')
    [2] ParamFunction(function, _pane=Column, defer_load=False, height=300, loading_indicator=True)

In [12]:
messages =  context.copy()
messages.append(
{'role':'system', 'content':'create a json summary of the previous food order. Itemize the price for each item\
 The fields should be 1) pizza, include size 2) list of toppings 3) list of drinks, include size   4) list of sides include size  5)total price '},
)
 #The fields should be 1) pizza, price 2) list of toppings 3) list of drinks, include size include price  4) list of sides include size include price, 5)total price '},

response = get_completion_from_messages(messages, temperature=0)
print(response)

```json
{
    "order": {
        "pizza": {
            "type": "pepperoni",
            "size": "medium",
            "price": 10.00
        },
        "toppings": [
            {
                "name": "extra cheese",
                "price": 2.00
            },
            {
                "name": "mushrooms",
                "price": 1.50
            }
        ],
        "drinks": [
            {
                "name": "coke",
                "size": "medium",
                "price": 2.00
            }
        ],
        "total_price": 18.50
    }
}
```
